Explore the dataset
==

**In this project:**

   **1. Processing medium-sized datasets using chunks**
    
   **2. Exploring and analyzing data using SQLite**
  

In [1]:
import pandas as pd
data_iter = pd.read_csv("crunchbase-investments.csv", chunksize=5000, 
                        encoding='latin1')
null_count_dict = {}
memory_use_dict = {}
iteration = 0
total_memory_use = 0

for chunk in data_iter:
    #Each column's missing value counts
    #Each column's memory footprint
    null_count = chunk.isnull().sum()
    memory_use = chunk.memory_usage(deep=True)
    total_use = memory_use.sum()
    total_memory_use += total_use
    for col in chunk.columns:
        if col not in null_count_dict:
            null_count_dict[col] = null_count[col]
            memory_use_dict[col] = memory_use[col]
        else:
            null_count_dict[col] += null_count[col]
            memory_use_dict[col] += memory_use[col]
    #Each column's default data type
    if iteration == 0:
        dtype_dict = chunk.dtypes
        iteration += 1
    

#format the result a little bit
def print_result(title,content):
    print("-"*len(title))
    print(title)
    print("-"*len(title))
    print(content)
    
print_result("Missing value counts",null_count_dict)
print_result("Default data type",dtype_dict)
print_result("Memory footprint",memory_use_dict)
print_result("Total memory footprint",total_memory_use/(1024**2))

--------------------
Missing value counts
--------------------
{'company_region': 1, 'company_city': 533, 'investor_region': 2, 'investor_state_code': 16809, 'funding_round_type': 3, 'funded_month': 3, 'investor_name': 2, 'investor_country_code': 12001, 'company_category_code': 643, 'funded_quarter': 3, 'funded_year': 3, 'raised_amount_usd': 3599, 'company_state_code': 492, 'company_name': 1, 'company_country_code': 1, 'company_permalink': 1, 'funded_at': 3, 'investor_city': 12480, 'investor_category_code': 50427, 'investor_permalink': 2}
-----------------
Default data type
-----------------
company_permalink          object
company_name               object
company_category_code      object
company_country_code       object
company_state_code         object
company_region             object
company_city               object
investor_permalink         object
investor_name              object
investor_category_code     object
investor_country_code      object
investor_state_code        

**Observe the dataset**

In [2]:
data_five = pd.read_csv("crunchbase-investments.csv", encoding='latin1', nrows=5)
print(data_five)

     company_permalink company_name company_category_code  \
0    /company/advercar     AdverCar           advertising   
1  /company/launchgram   LaunchGram                  news   
2        /company/utap         uTaP             messaging   
3    /company/zoopshop     ZoopShop              software   
4    /company/efuneral     eFuneral                   web   

  company_country_code company_state_code         company_region  \
0                  USA                 CA                 SF Bay   
1                  USA                 CA                 SF Bay   
2                  USA                NaN  United States - Other   
3                  USA                 OH               Columbus   
4                  USA                 OH              Cleveland   

    company_city          investor_permalink      investor_name  \
0  San Francisco  /company/1-800-flowers-com  1-800-FLOWERS.COM   
1  Mountain View        /company/10xelerator        10Xelerator   
2            NaN       

*Observations:*

*1. Some columns aren't useful for analysis (investor_permalink).*

*2. funded_at column can be splited into three columns (funded_year, funded_month, funded_day), and then funded_at and funded_month column can be deleted.*

*3. Keep the quarter only in funded_quarter column.*

Optimizing data types
==

In [3]:
data_iter = pd.read_csv("crunchbase-investments.csv", chunksize=5000, 
                        encoding='latin1')
#Identify text columns
text_col_list = dtype_dict[dtype_dict=="object"].keys().tolist()
print_result("Text column names", text_col_list)

#Identify text columns which can be optimized
text_count_dict = {}
combined_text_count = {}
overall_text_count = {}
rows = 0
for chunk in data_iter:
    #calculate total rows
    rows += chunk.shape[0]
    # Identify text columns
    for key in text_col_list:
        if key in text_count_dict:
            text_count_dict[key].append(chunk[key].value_counts())
        else:
            text_count_dict[key] = [chunk[key].value_counts()]
for key in text_count_dict.keys():
    combined_text_count[key] = pd.concat(text_count_dict[key])
    overall_text_count[key] = combined_text_count[key].groupby(combined_text_count[key].index).sum()
####### print_result("Value count for each textcolumn", overall_text_count)
print_result("Total rows", rows)

# List names of the columns which have less than 10% unique values
to_cat_list = []
unique_values = {}
for key in overall_text_count:
    unique_values[key] = len(overall_text_count[key])
    if unique_values[key]/rows < 0.015:
        to_cat_list.append(key)
print_result("Unique values for each text column", unique_values)
print_result("Columns suitable for converting to catogorical", to_cat_list)

-----------------
Text column names
-----------------
['company_permalink', 'company_name', 'company_category_code', 'company_country_code', 'company_state_code', 'company_region', 'company_city', 'investor_permalink', 'investor_name', 'investor_category_code', 'investor_country_code', 'investor_state_code', 'investor_region', 'investor_city', 'funding_round_type', 'funded_at', 'funded_month', 'funded_quarter']
----------
Total rows
----------
52870
----------------------------------
Unique values for each text column
----------------------------------
{'company_region': 546, 'company_city': 1229, 'investor_region': 585, 'investor_state_code': 50, 'funding_round_type': 9, 'funded_month': 192, 'investor_name': 10465, 'investor_country_code': 72, 'funded_quarter': 72, 'company_category_code': 43, 'company_state_code': 50, 'company_name': 11573, 'company_country_code': 2, 'company_permalink': 11573, 'funded_at': 2808, 'investor_city': 990, 'investor_category_code': 33, 'investor_permalink

In [4]:
to_catogorical = ['company_category_code', 'funding_round_type', 
                  'investor_category_code', 'company_country_code', 
                  'company_region', 'investor_region', 'investor_country_code', 
                  'company_state_code', 'investor_state_code', 'funded_quarter']

In [5]:
data_iter = pd.read_csv("crunchbase-investments.csv", chunksize=5000, 
                        encoding='latin1')
iteration = 0
new_total_memory_use = 0

for chunk in data_iter:
    # Optimize the numerical columns
    ###### filter out null values
    chunk = chunk[chunk["funded_at"].notnull()]
    chunk["funded_day"] = [c.split("-")[2] for c in chunk["funded_at"]]
    chunk["funded_month"] = [c.split("-")[1] for c in chunk["funded_at"]]        
    int_col_list = ["raised_amount_usd", "funded_day", "funded_month", "funded_year"]
    for col in int_col_list :
        chunk[col] = pd.to_numeric(chunk[col], downcast='integer')
    # Optimize text columns
    chunk["funded_quarter"] = [c.split("-")[1] for c in chunk["funded_quarter"].astype("object")]
    for col in to_catogorical:
        chunk[col] = chunk[col].astype("category")
    #drop the columns that are not useful in analysis
    chunk.drop(['investor_permalink', 'funded_at'], axis=1, inplace=True)
    new_total_memory_use += chunk.memory_usage().sum()
    iteration+=1
    
print_result("New total memory footprint",new_total_memory_use/(1024**2))

--------------------------
New total memory footprint
--------------------------
3.64965438843


**Memory usage is dramatically dropped from 56.98 MB to 3.64 MB!**

Loading Chunks Into SQLite
==

In [6]:
import sqlite3
conn = sqlite3.connect("investments.db")

data_iter = pd.read_csv("crunchbase-investments.csv", chunksize=5000, 
                        encoding='latin1')
itera = 0
for chunk in data_iter:
    # Optimize the numerical columns
    ###### filter out null values
    chunk = chunk[chunk["funded_at"].notnull()]
    chunk["funded_day"] = [c.split("-")[2] for c in chunk["funded_at"]]
    chunk["funded_month"] = [c.split("-")[1] for c in chunk["funded_at"]]
    int_col_list = ["funded_day", "funded_month", "funded_year", "raised_amount_usd"]
    for col in int_col_list :
        chunk[col] = pd.to_numeric(chunk[col], downcast='integer')
    chunk["raised_amount_usd"] = pd.to_numeric(chunk["raised_amount_usd"].astype("object"), downcast='integer')
    # Optimize text columns
    chunk["funded_quarter"] = [c.split("-")[1] for c in chunk["funded_quarter"]]
    for col in to_catogorical:
        chunk[col] = chunk[col].astype("category")
    #drop the columns that are not useful in analysis
    chunk = chunk.drop(['investor_permalink', 'funded_at', 'company_permalink'], axis=1)
    if itera == 0:
        new_dtype_dict = chunk.dtypes
        itera += 1
    chunk.to_sql("cr_investment", conn, if_exists="append")
print_result("New data types", new_dtype_dict)
pd.read_sql("PRAGMA table_info(cr_investment)", conn)

--------------
New data types
--------------
company_name                object
company_category_code     category
company_country_code      category
company_state_code        category
company_region            category
company_city                object
investor_name               object
investor_category_code    category
investor_country_code     category
investor_state_code       category
investor_region           category
investor_city               object
funding_round_type        category
funded_month                  int8
funded_quarter            category
funded_year                  int16
raised_amount_usd          float64
funded_day                    int8
dtype: object


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,company_name,TEXT,0,None,0
2,2,company_category_code,TEXT,0,None,0
3,3,company_country_code,TEXT,0,None,0
4,4,company_state_code,TEXT,0,None,0
5,5,company_region,TEXT,0,None,0
6,6,company_city,TEXT,0,None,0
7,7,investor_name,TEXT,0,None,0
8,8,investor_category_code,TEXT,0,None,0
9,9,investor_country_code,TEXT,0,None,0


Data Exploration And Analysis
==

**What proportion of the total amount of funds did the top 10% raise? What about the top 1%? How about the bottom 10% and bottom 1%?**

In [7]:
sum_top_10_per = '''SELECT SUM(selected)
FROM (SELECT raised_amount_usd AS selected
FROM (SELECT raised_amount_usd FROM cr_investment 
WHERE raised_amount_usd IS NOT NULL)
ORDER BY raised_amount_usd DESC
LIMIT (SELECT ROUND(0.01*COUNT(*)) AS tem_per FROM cr_investment));
'''

sum_top_1_per = '''SELECT SUM(selected)
FROM (SELECT raised_amount_usd AS selected
FROM (SELECT raised_amount_usd FROM cr_investment 
WHERE raised_amount_usd IS NOT NULL) 
ORDER BY raised_amount_usd DESC 
LIMIT (SELECT ROUND(0.001*COUNT(*)) AS tem_per FROM cr_investment));
'''

sum_bot_10_per = '''SELECT SUM(selected)
FROM (SELECT raised_amount_usd AS selected
FROM (SELECT raised_amount_usd FROM cr_investment 
WHERE raised_amount_usd IS NOT NULL) 
ORDER BY raised_amount_usd
LIMIT (SELECT ROUND(0.01*COUNT(*)) AS tem_per FROM cr_investment));
'''

sum_bot_1_per = '''SELECT SUM(selected)
FROM (SELECT raised_amount_usd AS selected
FROM (SELECT raised_amount_usd FROM cr_investment 
WHERE raised_amount_usd IS NOT NULL) 
ORDER BY raised_amount_usd
LIMIT (SELECT ROUND(0.001*COUNT(*)) AS tem_per FROM cr_investment));
'''

print_result("Total fund amount - Top 10%",
             pd.read_sql(sum_top_10_per, conn)['SUM(selected)'][0])
print_result("Total fund amount - Top 1%",
             pd.read_sql(sum_top_1_per, conn)['SUM(selected)'][0])
print_result("Total fund amount - Bottom 10%",
             pd.read_sql(sum_bot_10_per, conn)['SUM(selected)'][0])
print_result("Total fund amount - Bottom 1%",
             pd.read_sql(sum_bot_1_per, conn)['SUM(selected)'][0])


---------------------------
Total fund amount - Top 10%
---------------------------
540874790708.0
--------------------------
Total fund amount - Top 1%
--------------------------
230616000000.0
------------------------------
Total fund amount - Bottom 10%
------------------------------
40223396.0
-----------------------------
Total fund amount - Bottom 1%
-----------------------------
1560000.0


**Which category of company attracted the most investments?**

In [8]:
most_attractive_cat = '''
SELECT company_category_code, SUM(raised_amount_usd), COUNT(company_category_code)
FROM (SELECT * FROM cr_investment WHERE raised_amount_usd IS NOT NULL)
GROUP BY company_category_code
ORDER BY SUM(raised_amount_usd) DESC
LIMIT 1
'''
print_result("Company category - Most investments",
             pd.read_sql(most_attractive_cat, conn))

-----------------------------------
Company category - Most investments
-----------------------------------
  company_category_code  SUM(raised_amount_usd)  COUNT(company_category_code)
0               biotech            4.415857e+11                         19440


**Which investor contributed the most money (across all startups)? Which investors contributed the most money per startup?**

In [9]:
most_investor_sum = '''
SELECT investor_name, SUM(raised_amount_usd), COUNT(investor_name)
FROM (SELECT * FROM cr_investment WHERE raised_amount_usd IS NOT NULL)
GROUP BY investor_name
ORDER BY SUM(raised_amount_usd) DESC
LIMIT 1
'''
most_investor_per = '''
SELECT investor_name, raised_amount_usd
FROM (SELECT * FROM cr_investment WHERE raised_amount_usd IS NOT NULL)
ORDER BY raised_amount_usd DESC
LIMIT 1
'''
print_result("Most contributed investor - All startup",
             pd.read_sql(most_investor_sum, conn))
print_result("Most contributed investor - Per startup",
             pd.read_sql(most_investor_per, conn))

---------------------------------------
Most contributed investor - All startup
---------------------------------------
                      investor_name  SUM(raised_amount_usd)  \
0  Kleiner Perkins Caufield & Byers            4.487131e+10   

   COUNT(investor_name)  
0                  1500  
---------------------------------------
Most contributed investor - Per startup
---------------------------------------
  investor_name  raised_amount_usd
0   BrightHouse       3.200000e+09


**Which funding round was the most popular? Which was the least popular?**

In [10]:
most_popular_round = '''
SELECT funding_round_type, SUM(raised_amount_usd), COUNT(funding_round_type)
FROM (SELECT * FROM cr_investment WHERE raised_amount_usd IS NOT NULL)
GROUP BY funding_round_type
ORDER BY SUM(raised_amount_usd) DESC
LIMIT 1
'''
print_result("Most popular round",
             pd.read_sql(most_popular_round, conn))

------------------
Most popular round
------------------
  funding_round_type  SUM(raised_amount_usd)  COUNT(funding_round_type)
0          series-c+            1.063014e+12                      43056
